# GPT-SoVITS Infer

## Env Setup (Run Once Only)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%writefile /content/setup.sh
set -e

cd /content

git clone https://github.com/ljgit428/GPT-SoVITS

cd GPT-SoVITS

mkdir -p GPT_weights
mkdir -p SoVITS_weights

if conda env list | awk '{print $1}' | grep -Fxq "GPTSoVITS"; then
    :
else
    conda create -n GPTSoVITS python=3.10 -y
fi

source activate GPTSoVITS

pip install ipykernel

bash install.sh --device CU126 --source HF

Writing /content/setup.sh


### 2.

In [3]:
%pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh")
!cd /content && bash setup.sh

⏬ Downloading https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:01:55
🔁 Restarting kernel...
Cloning into 'GPT-SoVITS'...
remote: Enumerating objects: 5284, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 5284 (delta 41), reused 11 (delta 11), pack-reused 5217 (from 3)
Receiving objects: 100% (5284/5284), 13.50 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (3089/3089), done.
Channels:
 - defaults
Platform: linux-64
Solving environment: | done

## Package Plan ##

  environment location: /usr/local/envs/GPTSoVITS

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.11.4  |       h06a4308_0         128 KB
    expat-2.7.3                |       h3385a95_0 

In [1]:
!mkdir -p /content/GPT-SoVITS/reference_audio

!cp -f "/content/drive/MyDrive/GPT_weights/Seia-e15.ckpt" "/content/GPT-SoVITS/GPT_weights/"
!ls -lh /content/GPT-SoVITS/GPT_weights/Seia-e15.ckpt

!cp -f "/content/drive/MyDrive/SoVITS_weights/Seia_e8_s240.pth" "/content/GPT-SoVITS/SoVITS_weights/"
!ls -lh /content/GPT-SoVITS/SoVITS_weights/Seia_e8_s240.pth

!cp -f "/content/drive/MyDrive/reference_audio/ref.wav" "/content/GPT-SoVITS/reference_audio/"
!ls -lh /content/GPT-SoVITS/reference_audio/ref.wav

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
-rw------- 1 root root 149M Nov 26 08:29 /content/GPT-SoVITS/GPT_weights/Seia-e15.ckpt
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
-rw------- 1 root root 165M Nov 26 08:29 /content/GPT-SoVITS/SoVITS_weights/Seia_e8_s240.pth
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
-rw------- 1 root root 452K Nov 26 08:29 /content/GPT-SoVITS/reference_audio/ref.wav


In [2]:
import yaml

config_path = '/content/GPT-SoVITS/GPT_SoVITS/configs/tts_infer.yaml'

my_gpt_path = 'GPT_weights/Seia-e15.ckpt'
my_sovits_path = 'SoVITS_weights/Seia_e8_s240.pth'

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

config['custom']['t2s_weights_path'] = my_gpt_path
config['custom']['vits_weights_path'] = my_sovits_path

config['custom']['device'] = 'cuda'
config['custom']['is_half'] = True

config['custom']['version'] = 'v2ProPlus'

with open(config_path, 'w', encoding='utf-8') as f:
    yaml.dump(config, f, allow_unicode=True)


In [3]:
!pip install pyngrok

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
from IPython.display import Audio
Audio("/content/GPT-SoVITS/reference_audio/ref.wav")
#必要なものはありますか？どうぞおっしゃってみてください。
# Is there anything you need? Please feel free to ask.

In [5]:
import os

os.environ["GS_REF_AUDIO"] = "/content/GPT-SoVITS/reference_audio/ref.wav"

os.environ["GS_PROMPT_TEXT"] = "必要なものはありますか？どうぞおっしゃってみてください。"

os.environ["GS_PROMPT_LANG"] = "all_ja"

print("Config ready!")
print(f"current default audio: {os.environ['GS_REF_AUDIO']}")
print(f"current default text: {os.environ['GS_PROMPT_TEXT']}")


Config ready!
current default audio: /content/GPT-SoVITS/reference_audio/ref.wav
current default text: 必要なものはありますか？どうぞおっしゃってみてください。


In [ ]:
import os
from pyngrok import ngrok

# ==========================================
# 1. Set up Ngrok Token
# ==========================================
NGROK_TOKEN = "NGROK_TOKEN"

# Configure the Token
!ngrok config add-authtoken {NGROK_TOKEN}

# ==========================================
# 2. Start Ngrok Tunnel (Start tunnel first to get the address)
# ==========================================
# Kill old processes to prevent conflicts
ngrok.kill()

# Open a tunnel on port 9880
try:
    # Force bind to localhost to avoid IPv6 issues
    public_url = ngrok.connect("127.0.0.1:9880").public_url
    print("\nAPI service is about to start!")
    print(f"API Public Address: {public_url}")
    print(f"Please copy the address above and update the Django .env file")
except Exception as e:
    print("Ngrok failed to start:", e)

# ==========================================
# 3. Start GPT-SoVITS (Run in foreground to display logs)
# ==========================================
print("\n Starting inference service, please wait for model loading (approx. 1-2 mins)...")
print("----------------------------------------------------------------")

!cd /content/GPT-SoVITS && \
 source activate GPTSoVITS && \
 pip install torchcodec && \
 pip install transformers==4.36.2 peft==0.7.1 && \
 python api_v2_colab.py -a 127.0.0.1 -p 9880 -c GPT_SoVITS/configs/tts_infer.yaml

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

API service is about to start!
API Public Address: https://unimplied-annett-peakishly.ngrok-free.dev
Please copy the address above and update the Django .env file

 Starting inference service, please wait for model loading (approx. 1-2 mins)...
----------------------------------------------------------------
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.6 MB/s  0:00:00


t=2025-11-26T08:29:16+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.2 MB/s  0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0
  Attempting uninstall: peft
    Found existing installation: peft 0.18.0
    Uninstalling peft-0.18.0:
      Successfully uninstalled peft-0.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [peft]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faster-whisper 1.2.1 requires onnxruntime<2,>=1.14, which is not installed.
/usr/local/envs/GPTSoVITS/lib/python3.10/site-pac

In [ ]:
# check root
!ls -lh /content/GPT-SoVITS/ref.wav

!ls -lh /content/GPT-SoVITS/reference_audio/ref.wav